In [2]:
import pm4py
import graphviz
import pandas as pd
import numpy as np

#### 데이터 준비

In [3]:
xes_prod = pm4py.read_xes(r'C:\PROJECT\0_중견\Dataset\Production.xes')

c:\Users\User\anaconda3\envs\basic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 225/225 [00:00<00:00, 687.32it/s]


In [4]:
csv_prod = pd.DataFrame(xes_prod)

### 최종 데이터

In [5]:
Log_prod = pm4py.format_dataframe(csv_prod,case_id='case:concept:name',activity_key='concept:name',timestamp_key='Start Timestamp')

C:\Users\User\AppData\Local\Temp\ipykernel_18236\213123305.py:1: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  Log_prod = pm4py.format_dataframe(csv_prod,case_id='case:concept:name',activity_key='concept:name',timestamp_key='Start Timestamp')


### Event Log 복잡성 평가 지표

###### 1. #_Cases --> Case의 수
###### 2. Avg_Trace_Len_Cases -->  각 Case들의 평균 Trace길이
###### 3. #_distinct_Activ --> Activity의 종류 갯수
###### 4. Avg#_dist_act_per_Case --> 중복 제외 각 Case의 서로다른 Activity 종류 갯수
###### 5. Avg_set_based_non_overlap_of_Traces --> 각 Trace의 쌍을 Overlap 기반 비교한 값 = Trace가 모두 같은 Activity로 이루어져있으면 0
###### 6. #_distinct_Case -->  서로 다른 Trace의 갯수
###### 7. #_Events -->  Event Log에 있는 총 Event의 갯수 = 전체 Case의 수 X Case 평균 Trace길이
###### 8. #_direct_successions --> 서로 다른 Direct Succession관계의 갯수
###### 9. #_Start_Activ --> Start Activity의 갯수
###### 10. #_End_Activ -->  End Activity의 갯수


In [6]:
from itertools import *

# 복잡성 지표 함수

#### pm4py, pandas, itertools 필요

In [7]:
def event_log_check(LogData,Key_Case,Key_Activity,Key_Timestamp):
    # Event Log 복잡성 평가 Metrics
    EL_Metrics=pd.DataFrame(index=range(0,10),columns=['Indicator','Value'])
    EL_Metrics['Indicator'] = ['#_ Cases','Avg_Trace_Len_Cases',
                               '#_distinct_Activ',
                               'Avg#_dist_act_per_Case',
                               'Avg_set_based_non_overlap_of_Traces',
                               '#_distinct_Case',
                               '#_Events','#_direct_successions',
                               '#_Start_Activ',
                               '#_End_Activ']


    #Trace와 해당 Trace를 가지는 케이스의 수 (key: Trace, Value = Case수)
    dict_trace = pm4py.get_variants(LogData,
                                    activity_key= Key_Activity,
                                    timestamp_key= Key_Timestamp,
                                    case_id_key= Key_Case)
    


    # Case 갯수
    EL_Metrics.iloc[0,1]=LogData[Key_Case].nunique()


    # 각 Case 평균 Trace 길이
    df_trace = pd.DataFrame()
    df_trace['trace'] = dict_trace.keys()
    df_trace['CaseNum'] = dict_trace.values()
    sum_trace=0
    for i in range(len(df_trace)):
        sum_trace = sum_trace + len(df_trace.iloc[i,0])*df_trace.iloc[i,1]
    EL_Metrics.iloc[1,1] = sum_trace/sum(df_trace['CaseNum'])


    # 액티비티의 종류의 수
    # #_distinct_Activity
    EL_Metrics.iloc[2,1] = LogData[Key_Activity].nunique()


    # Avg num of distinct activities per case
    # 각 케이스 중복 제외 서로 다른 액티비티 갯수 평균
    distinct_sum = 0
    for i in df_trace['trace']:
        distinct_sum = distinct_sum + len(set(i))
    EL_Metrics.iloc[3,1] = distinct_sum/sum(df_trace['CaseNum'])


    # Avg set based non overlap of Traces
    sbnot_sum = 0
    for i in combinations(df_trace['trace'],2):
        casenum1=df_trace[df_trace['trace']==i[0]]
        casenum2=df_trace[df_trace['trace']==i[1]]  # 트레이스의 케이스 수
        tr_casenum = casenum1.iloc[0,1] * casenum2.iloc[0,1]
        tr1 = list(i[0])
        tr2 = list(i[1])
        sumlen = len(tr1+tr2) #합집합 길이
        same_trace = list(set(tr1)&set(tr2))
        interlen = len(same_trace) #교집합 길이
        sbnot_sum = sbnot_sum + (tr_casenum * (interlen/sumlen))
        sbnot_Log = 1 - sbnot_sum/(sum(df_trace['CaseNum']))**2
        sbnot_sum
    EL_Metrics.iloc[4,1] = sbnot_Log


    # num of distinct_Case
    EL_Metrics.iloc[5,1] = len(df_trace['CaseNum'])


    # num of Events
    EL_Metrics.iloc[6,1] = EL_Metrics.iloc[0,1]*EL_Metrics.iloc[1,1]


    # num of Direct Successions
    dict_dfg = pm4py.discover_performance_dfg(LogData,
                                              case_id_key=Key_Case,
                                              activity_key=Key_Activity,
                                              timestamp_key=Key_Timestamp)
    df_dfg = pd.DataFrame()
    df_dfg['dfg']=dict_dfg[0].keys()
    EL_Metrics.iloc[7,1] = df_dfg['dfg'].nunique()


    # num of Start Activities
    EL_Metrics.iloc[8,1] = len(pm4py.get_start_activities(LogData,
                                                      activity_key= Key_Activity,
                                                      timestamp_key= Key_Timestamp,
                                                      case_id_key= Key_Case))


    # num of End Activities
    EL_Metrics.iloc[9,1] = len(pm4py.get_end_activities(LogData,
                                                      activity_key= Key_Activity,
                                                      timestamp_key= Key_Timestamp,
                                                      case_id_key= Key_Case))

    print(EL_Metrics)

In [12]:
log_check = event_log_check(Log_prod,'case:concept:name','concept:name','Start Timestamp')
log_check

                             Indicator      Value
0                             #_ Cases        225
1                  Avg_Trace_Len_Cases  20.191111
2                     #_distinct_Activ         55
3               Avg#_dist_act_per_Case       6.04
4  Avg_set_based_non_overlap_of_Traces   0.956476
5                      #_distinct_Case        221
6                             #_Events     4543.0
7                 #_direct_successions        381
8                        #_Start_Activ         31
9                          #_End_Activ         21
